In [ ]:
# Задача: заботать Reinforcement learning на примере машинки, которая должна собрать все шарики с поля в свой склад на
# определенном поле. Машинка может совершать 6 действий (4 движения, поднятие и бросание шарика)
# Реализован алгоритм Q-learning. Выяснено, что он не является стабильным. В будущем планируется применение
# алгоритма Deep Q-learning с расширением количества действий машинки и увеличением количества машинок на поле.

In [37]:
import math
import numpy as np
import random

## Global parameters

In [38]:
#  Глобальные параметры  #
Tmax = 500 #количество времени, отведенное машине на работу
dt = 1 # время, затрчиваемое на 1 действие
nactions = 6 #количество действий
mymap = np.array([('0','1','0'),('0','0','1'),('0','1','0')]) #разбрасываем на карте 3 мячика
print(mymap)

[['0' '1' '0']
 ['0' '0' '1']
 ['0' '1' '0']]


## Определение машинки: возможные действия и внутренние параметры

In [39]:
class Machine:

    def __init__(self): #инициализация машины
        global nactions, mymap
        
        self.velocity = np.array([1,1]) #скорость перемещения (на сколько клеточек за 1 ход может переместиться по Х и У)
        self.maxcapacity = 1 #максимальная вместимость (сколько шариков может перевозить)
        self.capacity = self.maxcapacity #вместимость кузова машины в определенный момент
        
        x0 = 0
        y0 = 0
        self.position = np.array([x0,y0])  #начальные координаты машины
        
        xs = 1
        ys = 1
        self.stockposition = np.array([xs,ys]) #координаты личного склада машины.
        
        self.state = y0*nactions+x0 #состояние трактора в данный момент времени
        
        self.Q = np.random.rand(2*mymap.shape[0]*mymap.shape[0],nactions) # инициализация матрицы весов действий
        
    def getAction(self): #выбор одного действия из возможных в данном положении
        global Tmax,mymap
        
        if (self.position[0] == 0): 
            self.Q[self.state][0] = -10 # - на каждой итерации нужно проверять возможные действия,
            #так как карта может меняться другой машиной (в будущем)
            
        if (self.position[1] == 0):
            self.Q[self.state][2] = -10
            
        if (self.position[0] == np.shape(mymap)[0]-1):
            self.Q[self.state][1] = -10
            
        if (self.position[1] == np.shape(mymap)[1]-1):
            self.Q[self.state][3] = -10

        if (self.capacity == 0) or (mymap[self.position[0]][self.position[1]] == '0'):
            self.Q[self.state][4] = -10
            
        if (self.capacity == self.maxcapacity):
            self.Q[self.state][5] = -10   
        e = math.exp(-t/0.1) #вероятность принятия рандомного решения
        x = np.random.randint(0,101,1)
        if (x >= (1.0-e)*100.0):
#        e = Tmax-t
#        x = random.uniform(0, Tmax)
#        if x<e:
            
            foo = self.Q[self.state]
            even = [] #массив возможных действий
            for i in range(len(foo)):
                if foo[i] > 0:
                    even.append(i)
#            print(even)
            action = random.choice(even) #рандомный выбор одного действия из массива возможных действий

        else:
            action = np.argmax(self.Q[self.state]) #осознанный выбор действия с наибольшим весом
        return action
    
    def run(self, direction): #метод перемещения машины в одном из 4х определенных направлений
        global dt
        self.position = self.position + self.velocity*dt*direction
          
    def pickup(self): #поднятие мяча в кузов (далее учтено, что если машина забирает мяч из своего склада, то 
                        # она получает отрицательную награду)
        global mymap
        self.capacity -= 1
        k = mymap[self.position[0]][self.position[1]]
        mymap[self.position[0]][self.position[1]] = str(int(k) - 1)
        
    def put(self): #бросание мяча
        global mymap
        self.capacity += 1
#        if self.position == self.stockposition #эта строчка закоменчена, чтобы дать возможность ему бросать, где захочет
        k = mymap[self.position[0]][self.position[1]]
        mymap[self.position[0]][self.position[1]] = str(int(k) + 1)
    
    def updateState(self,action): #функция переходов в следующее состояние при совершении действия (v=1)
        if action == 0: #движение вверх
            self.state = self.state - np.shape(mymap)[1]
        if action == 1: #движение вниз
            self.state = self.state + np.shape(mymap)[1]
        if action == 2: #движение влево
            self.state = self.state - 1
        if action == 3: #движение вправо
            self.state = self.state + 1
        if action == 4: #положить шарик в кузов
            self.state += mymap.shape[0]*mymap.shape[1]
        if action == 5: #скинуть 1 шарик из кузова
            self.state -= mymap.shape[0]*mymap.shape[1]
            
    def checkstock(self): #функция проверки количества мячей на складе в данный момент времени
         return int(mymap[self.stockposition[0]][self.stockposition[1]])
    
    def updateQ(self,action,award,prevstate): #обучение (обновление матрицы)
        alpha = 0.1
        gamma = 0.15
        self.Q[prevstate][action] = self.Q[prevstate][action] + alpha*(award + gamma*np.max(self.Q[self.state]) - self.Q[prevstate][action])
    
    def ImHere(self):
        print('Мои координаты: ', self.position, ', вместимость: ', self.capacity, '\n')    
            
    def oneStep(self): #один ход машины: выбор действия, совершение действия, получение награды, 
                 #обновление своего состояния, обновление веса в матрице предыдущего состояния с учетом полученной награды.
        choice = self.getAction()
        prevstate = self.state
        prevstock = self.checkstock()
        if (choice == 0): #движение вверх
            self.run((-1,0))
        if (choice == 1): #движение вниз
            self.run((1,0))        
        if (choice == 2): #движение влево
            self.run((0,-1))
        if (choice == 3): #движение вправо
            self.run((0,1))
        if (choice == 4): #положить шарик в кузов
            self.pickup()
        if (choice == 5): #скинуть 1 шарик из кузова
            self.put()
        newstock = self.checkstock()
        stepaward = 5*(newstock - prevstock)
        self.updateState(choice)
        self.updateQ(choice, stepaward, prevstate)
#        self.ImHere()

In [40]:
m = Machine()
while (mymap[1][1] != '3'):
    mymap = np.array([('0','1','0'),('0','0','1'),('0','1','0')])
    m = Machine()
    print('Начальный вид карты: \n',mymap)
    print()
    for t in range (Tmax):
        m.oneStep()
    print('Вид карты после', Tmax, 'действий машины: \n',mymap)
print('\n')
#print(m.Q)

Начальный вид карты: 
 [['0' '1' '0']
 ['0' '0' '1']
 ['0' '1' '0']]

Вид карты после 500 действий машины: 
 [['0' '0' '0']
 ['1' '2' '0']
 ['0' '0' '0']]
Начальный вид карты: 
 [['0' '1' '0']
 ['0' '0' '1']
 ['0' '1' '0']]

Вид карты после 500 действий машины: 
 [['0' '0' '3']
 ['0' '0' '0']
 ['0' '0' '0']]
Начальный вид карты: 
 [['0' '1' '0']
 ['0' '0' '1']
 ['0' '1' '0']]

Вид карты после 500 действий машины: 
 [['0' '1' '0']
 ['1' '0' '1']
 ['0' '0' '0']]
Начальный вид карты: 
 [['0' '1' '0']
 ['0' '0' '1']
 ['0' '1' '0']]

Вид карты после 500 действий машины: 
 [['0' '0' '0']
 ['1' '2' '0']
 ['0' '0' '0']]
Начальный вид карты: 
 [['0' '1' '0']
 ['0' '0' '1']
 ['0' '1' '0']]

Вид карты после 500 действий машины: 
 [['0' '0' '0']
 ['0' '1' '0']
 ['0' '0' '2']]
Начальный вид карты: 
 [['0' '1' '0']
 ['0' '0' '1']
 ['0' '1' '0']]

Вид карты после 500 действий машины: 
 [['1' '0' '0']
 ['1' '0' '1']
 ['0' '0' '0']]
Начальный вид карты: 
 [['0' '1' '0']
 ['0' '0' '1']
 ['0' '1' '0']]

